In [6]:
import sys 
from pathlib import Path

#Add the project root to the Python path
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

In [7]:
import pandas as pd
from src.data import fetch_yahoo

symbol = "SPY"
start, end = "2016-01-01", "2026-01-01"

df = fetch_yahoo(symbol, start, end)
df.head(), df.tail(), df.isna().sum()


(Price             Open        High         Low       Close     Volume
 Ticker             Spy         Spy         Spy         Spy        Spy
 Date                                                                 
 2016-01-04  169.922983  170.380648  168.312652  170.372177  222353500
 2016-01-05  170.694205  171.117974  169.550036  170.660309  110845800
 2016-01-06  168.100756  169.558523  167.473586  168.507584  152112600
 2016-01-07  165.549677  167.337983  164.074955  164.464828  213436100
 2016-01-08  165.431031  165.990409  162.371416  162.659576  209817200,
 Price             Open        High         Low       Close    Volume
 Ticker             Spy         Spy         Spy         Spy       Spy
 Date                                                                
 2025-12-24  687.950012  690.830017  687.799988  690.380005  39445600
 2025-12-26  690.640015  691.659973  689.270020  690.309998  41613300
 2025-12-29  687.539978  689.200012  686.070007  687.849976  62559500
 2025-12-30